<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

### Loading the Robot class
Before start programming for JetBot, we need to import the "Robot" class. This class allows us to easily control the JetBot motor

In [1]:
from jetbot import Robot

Robot class has been loaded, we can initialize this instance with the following statement.

In [2]:
robot = Robot()

Before we tried this example, we tried to rotate the motors on the left and right sides. Next we use more methods to control Jetbot.

In [3]:
robot.down(1)#PTZ rise

In [4]:
robot.up(1)#PTZ decline

In [8]:
robot.vertical_motors_stop()#stop PTZ

In [9]:
robot.forward(1)#Jetbot Robot car advance

In [10]:
robot.backward(1)#Jetbot Robot car back

In [11]:
robot.left(0.75)#Jetbot turn left

In [14]:
robot.right(0.75)#Jetbot turn right

In [15]:
robot.stop()#stop Jetbot

In [16]:
robot.set_bln(1)#Set Jetbot's onboard breathing lamp brightness to 1 (value range: 0 - 1.0)

If we need to move the robot for a certain period of time, we can use Python's time package. Execute the following code to load the time module.

In [17]:
import time

In [18]:
robot.left(0.5)
time.sleep(0.5)
robot.stop()

### Individually controlling the motor
Set the speed of each motor separately:
The first method is to call set_motors. For example, turn left for one second. We can set the left motor speed to 30% and the right motor to 60%, which will achieve different arc steering modes.

In [ ]:
robot.set_motors(0.5, 0.5)
time.sleep(1.0)
robot.stop()

In [ ]:
robot.left_motor.value = 0.3
robot.right_motor.value = 0.6
time.sleep(1.0)
robot.left_motor.value = 0.0
robot.right_motor.value = 0.0

### Using the traitlets library to connect to the Jupyter lab HTML control to operate the motor
Next we learn how to make some small graphical buttons (controls) on the Jupyter Notbooks page, then use traitlets to connect these widgets for control. In this way, we can control the car through the buttons on the web page.
First, we need to create and display two sliders for controlling the motor.

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display

# create two sliders with range [-1.0, 1.0]
left_slider = widgets.FloatSlider(description='left', min=-1.0, max=1.0, step=0.01, orientation='vertical')
right_slider = widgets.FloatSlider(description='right', min=-1.0, max=1.0, step=0.01, orientation='vertical')

# create a horizontal box container to place the sliders next to eachother
slider_container = widgets.HBox([left_slider, right_slider])

# display the container in this cell's output
display(slider_container)

After running the above code, you can see two vertical sliders on it.

Tips:
In the Jupyter Lab, you can actually pop the cell into another window, such as the two sliders. Although not in the same window, it is still connected to this notebook. The specific operation is to move the mouse to the right button of the cell (for example: slider), select "Creat new view for output", and then drag the window to the place you are satisfied.

Try clicking and dragging the slider up and down to see the change in value.

In [ ]:
import traitlets
left_link = traitlets.link((left_slider, 'value'), (robot.left_motor, 'value'))
right_link = traitlets.link((right_slider, 'value'), (robot.right_motor, 'value'))

You can try dragging the slider (slowly drag it first to prevent your Jetbot from suddenly rushing out of the boundary and causing damage), and you can see that the servo attached to the camera is spinning.
The link function we created above actually creates a two-way link!
That means that if we set the motor value elsewhere, the slider will be updated!
For example, the code shown below:

In [ ]:
robot.forward(0.5)
time.sleep(1.0)
robot.stop()

After running the above code, you can see that the slider has also changed, and the speed value of the motor will changed. If we want to disconnect this connection, we can call the unlink method to disconnect one by one.

In [ ]:
left_link.unlink()
right_link.unlink()

If we don't want a two-way connection, for example, we only want to use the slider to display the speed value of the motor, and do not want to control it, then to achieve this function, we can use the dlink function, the source on the left, On the right is the target, (the data comes from the motor and then on the target).

In [ ]:
left_link = traitlets.dlink((robot.left_motor, 'value'), (left_slider, 'value'))
right_link = traitlets.dlink((robot.right_motor, 'value'), (right_slider, 'value'))

### Add a function to an event
Another method to use traitlets is that attach a function to an event (for example, forward). As soon as the object changes, the function is called and some information that has changed is passed, such as the old and new values.
Create some buttons for controlling the robot to display on the notebook:

In [ ]:
# Create button
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# Display button
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

After executing the above code, you can see a set of robot control buttons shown above, but now you don't have any reaction when you click the button.

If we want to start control, we need to create some functions attached to the button on_click event.

In [ ]:
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.8)
    time.sleep(0.5)
    robot.stop()

def step_backward(change):
    robot.backward(0.8)
    time.sleep(0.5)
    robot.stop()

def step_left(change):
    robot.left(0.6)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.6)
    time.sleep(0.5)
    robot.stop()

We have defined those functions, we will attach these functions to the on_click event of each button.

In [ ]:
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

After executing the above code,
you will can control the Jetbot forward, backward, left turn, right turn, 
stop by clicking the button component we created above.

#### Heartbeat switch
The following code block is about how to use the 'heartbeat' package to stop Jetbot's movement.

This is a way to check if the connection between Jetbot and the browser still exists. You can adjust the heartbeat period (in seconds) by executing the slider after the code is executed. If the heartbeat cannot communicate back and forth between browsers, the heartbeat 'status' attribute value will be Will be set to dead, once the connection is restored, the status property will be set to alive.

In [ ]:
from jetbot import Heartbeat

heartbeat = Heartbeat()

# this function will be called when heartbeat 'alive' status changes
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        robot.stop()
        
heartbeat.observe(handle_heartbeat_status, names='status')

period_slider = widgets.FloatSlider(description='period', min=0.001, max=0.5, step=0.01, value=0.5)
traitlets.dlink((period_slider, 'value'), (heartbeat, 'period'))

display(period_slider, heartbeat.pulseout)